Get fractions for demographics

In [1]:
%load_ext autoreload
%autoreload 2


In [4]:
import sys
sys.path.append('../utils')
import wd_management
wd_management.set_wd_root()

called set_wd_root. current working directory is /Users/alexanderweinstein/Documents/Data_Products/db-equitable-development-tool/notebooks
changing wd to parent


In [5]:

import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

survey_package = rpackages.importr("survey")
base = rpackages.importr("base")

from rpy2.robjects import r, pandas2ri

pandas2ri.activate()

In [6]:
from ingest.load_data import load_data

PUMS = load_data(
            PUMS_variable_types=['demographics'],
            limited_PUMA=True,
            year=2019,
            requery=False,
        )["PUMS"]

In [7]:
PUMS['wnh'] = ((PUMS['RAC1P'] == 'White alone') & (PUMS['HISP'] =='Not Spanish/Hispanic/Latino')).astype(int)
PUMS['wnh']

person_id
20150000144861    1
20150000225401    1
20150000267661    1
20150000267662    1
20150000268321    1
                 ..
2019HU13854934    1
2019HU13891321    0
2019HU13891322    0
2019HU13958461    0
2019HU13970981    1
Name: wnh, Length: 33576, dtype: int64

In [6]:
rw_cols = [f"PWGTP{x}" for x in range(1, 81)]
weight_col = 'PWGTP'
geo_col = 'PUMA'
survey_design = survey_package.svrepdesign(
    variables=PUMS[["wnh"]],
    repweights=PUMS[rw_cols],
    weights=PUMS[weight_col],
    combined_weights=True,
    type="other",
    scale=4 / 80,
    rscales=1,
)


In [7]:
aggregated = survey_package.svyby(
    formula=PUMS["wnh"],
    by=PUMS[[geo_col]],
    design=survey_design,
    FUN=survey_package.svymean,
)

In [8]:
aggregated

,PUMA,V1,se
3701,3701,0.322707,0.007264
3801,3801,0.193707,0.005321
3901,3901,0.821303,0.006695
4001,4001,0.635467,0.006938
4101,4101,0.494444,0.006473


In [15]:
var = 'RAC1P'
categories = PUMS[var].unique()
all_fractions = pd.DataFrame(index=PUMS["PUMA"].unique())

for category in categories:
    PUMS[category] = (PUMS[var] == category).astype(int)
    survey_design = survey_package.svrepdesign(
    variables=PUMS[[category]],
    repweights=PUMS[rw_cols],
    weights=PUMS[weight_col],
    combined_weights=True,
    type="other",
    scale=4 / 80,
    rscales=1)
    single_fraction = survey_package.svyby(
        formula=PUMS[category],
        by=PUMS[[geo_col]],
        design=survey_design,
        FUN=survey_package.svymean,
    )
    single_fraction.drop(columns=['PUMA'])
    single_fraction.rename(columns={'V1':f'{category}-fraction', 'se':f'{category}-fraction-se'}, inplace=True)
    all_fractions = all_fractions.merge(single_fraction, left_index=True, right_index=True)
all_fractions

,PUMA_x,White alone-fraction,White alone-fraction-se,PUMA_y,Some Other Race alone-fraction,Some Other Race alone-fraction-se,PUMA_x,Black or African American alone-fraction,Black or African American alone-fraction-se,PUMA_y,...,"American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races-fraction-se",PUMA_y,Two or More Races-fraction,Two or More Races-fraction-se,PUMA_x,American Indian alone-fraction,American Indian alone-fraction-se,PUMA_y,Native Hawaiian and Other Pacific Islander alone-fraction,Native Hawaiian and Other Pacific Islander alone-fraction-se
4001,4001,0.745201,0.008315,4001,0.085335,0.006465,4001,0.059257,0.004639,4001,...,0.001379,4001,0.038191,0.004420,4001,0.001273,0.000638,4001,0.000443,0.000252
3701,3701,0.465471,0.010736,3701,0.268322,0.010740,3701,0.170533,0.010324,3701,...,0.001109,3701,0.043223,0.005093,3701,0.004065,0.001740,3701,0.002414,0.002053
4101,4101,0.632187,0.009074,4101,0.096920,0.007022,4101,0.076985,0.004489,4101,...,0.000886,4101,0.033864,0.002843,4101,0.000555,0.000420,4101,0.000457,0.000377
3801,3801,0.358764,0.009933,3801,0.382457,0.012172,3801,0.138972,0.006858,3801,...,0.000980,3801,0.079684,0.008974,3801,0.006496,0.001983,3801,0.000549,0.000394
3901,3901,0.911699,0.005684,3901,0.014155,0.002951,3901,0.009868,0.001877,3901,...,0.000524,3901,0.013700,0.002682,3901,0.000838,0.000564,3901,0.000674,0.000405
